<a href="https://colab.research.google.com/github/CSSamarasinghe/SE4050_Assignment/blob/IT21222740/%F0%9F%9B%925_class_Classification_Amazon_Reviews_%5BGRU_And_GLOVE_100d%5D_53____(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, GlobalMaxPool1D, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'amazon-reviews-for-sentianalysis-finegrained-csv:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2078107%2F3499094%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241003%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241003T164549Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D25a30a2ad799910d48f45f67793fb1d09f31560ae3773fa14870108018fa213261ae62aeb020867952a30d8094a21359c2546a6475193ac986bc2645a191d848c7223dbd5c68600aeb8f17d5a979be9750827328816b6743cf0d74ca79e5a28878f8606f4d3cd2176678d18b8a70a20bbf9be8037aa238d4b56bd1a0ba1a8d1fb4003bbd12f51fa6d438aaa8a7e8fa90a9291e62969f7df78c34dc6a71c76fb2f1c11e2964127136ec4d5554e9f1bdd7c530f8ddf987a08dd6cb91a849e4e98a2089149d22cb00ca687688c756071af4270e392fd79699c1f4a35ca16597523ca8bfe37fea11d5911465ea51d6eb17c1e3663da806763c8ef4e96e1f0a9ae9ab'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 654512809 bytes downloaded
Downloaded and uncompressed: amazon-reviews-for-sentianalysis-finegrained-csv
Data source import complete.


In [11]:
# Load dataset
data = pd.read_csv('../input/amazon-reviews-for-sentianalysis-finegrained-csv/amazon_review_fine-grained_5_classes_csv/train.csv')


In [12]:
data = data.sample(frac =0.2, random_state=42)
print(data.shape)

(600000, 3)


In [5]:
# Load GloVe embeddings
def load_glove_embeddings(file):
    words = {}
    with open(file, 'r') as f:
        for line in f:
            line = line.split(' ')
            words[line[0]] = np.array(line[1:], dtype=float)
    return words

In [6]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2024-10-04 17:19:10--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-10-04 17:19:10--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-10-04 17:19:10--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

gl

In [7]:
glove_embeddings = load_glove_embeddings('glove.6B.200d.txt')

In [13]:
# Initialize Lemmatizer
lemmatizer = WordNetLemmatizer()


In [14]:
# Tokenization and Lemmatization function
def message_to_token_list(s, words, tokenizer):
    tokens = tokenizer.tokenize(s)
    lowercased_tokens = [t.lower() for t in tokens]
    lemmatized_tokens = [lemmatizer.lemmatize(t) for t in lowercased_tokens]
    useful_tokens = [t for t in lemmatized_tokens if t in words]
    return useful_tokens

In [15]:
# Tokenize the reviews
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['review_text'])
sequences = tokenizer.texts_to_sequences(data['review_text'])

In [16]:
# Padding sequences
padded_sequences = pad_sequences(sequences, padding='post')

In [17]:
# Get labels
labels = data['class_index'] - 1  # Adjust class index to be 0-based for classification

In [62]:
from sklearn.model_selection import train_test_split

# Define the number of splits
n_splits = 10
X_test = np.array([])
y_test = np.array([])
# Create lists to hold the split datasets
X_train_splits = []
y_train_splits = []

X_train_val, X_test, y_train_val, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)



In [63]:
print(X_train_val.shape)
print(X_test.shape)
print(y_train_val.shape)
print(y_test.shape)

(480000, 224)
(120000, 224)
(480000,)
(120000,)


In [64]:
from sklearn.model_selection import train_test_split

# Split X_train and y_train into two separate training datasets
X_train_1, X_train_2, y_train_1, y_train_2 = train_test_split(X_train_val, y_train_val, test_size=0.5, random_state=42, stratify=y_train_val)

In [65]:
X_train_first, X_train_second, y_train_first, y_train_second = train_test_split(X_train_1, y_train_1, test_size=0.5, random_state=42, stratify=y_train_1)
X_train_third, X_train_fourth, y_train_third, y_train_fourth = train_test_split(X_train_2, y_train_2, test_size=0.5, random_state=42, stratify=y_train_2)
# Output the shapes to verify the split
print("X_train_1 shape:", X_train_1.shape)
print("y_train_1 shape:", y_train_1.shape)
print("X_train_2 shape:", X_train_2.shape)
print("y_train_2 shape:", y_train_2.shape)


X_train_1 shape: (240000, 224)
y_train_1 shape: (240000,)
X_train_2 shape: (240000, 224)
y_train_2 shape: (240000,)


In [68]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.2, random_state=42, stratify=y_test)

In [70]:
import pandas as pd

# Convert the NumPy array to a Pandas DataFrame
df = pd.DataFrame(X_train_first)
df1 = pd.DataFrame(X_train_second)

# Now you can use the head() method
print(df.head())
print(df1.head())

    0      1       2     3      4      5      6     7    8    9    ...  214  \
0  2403  24507  127044  5278   2288  40920     81     1  360   13  ...    0   
1    76   3284   12819    66    202     96    892  3284    2   47  ...    0   
2     3     13    7623  9005  48295   2018     98   530   75    3  ...    0   
3     3    733       9    65     75  10458  19069     8   21   28  ...    0   
4     1   1186     665    28    353     22   1141     1  101   13  ...    0   

   215  216  217  218  219  220  221  222  223  
0    0    0    0    0    0    0    0    0    0  
1    0    0    0    0    0    0    0    0    0  
2    0    0    0    0    0    0    0    0    0  
3    0    0    0    0    0    0    0    0    0  
4    0    0    0    0    0    0    0    0    0  

[5 rows x 224 columns]
     0      1    2      3     4     5     6      7     8     9    ...  214  \
0    143   1219  570   2515   577  3595    53    517   577   416  ...    0   
1  27791      4  228   2720   216     3   224    14

In [71]:
from tensorflow.keras.layers import Bidirectional

In [73]:

# Define the model with GloVe embeddings and a modified GRU architecture
sequence_length = X_train_first.shape
embedding_dim = 200  # GloVe embedding dimension
vocab_size = len(tokenizer.word_index) + 1  # Vocabulary size
embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Build the model
model = Sequential()

# Embedding layer initialized with GloVe embeddings, non-trainable (set trainable=False)
model.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_dim,
                    input_length= sequence_length,
                    weights=[embedding_matrix],
                    trainable=True))  # Set trainable=False to freeze GloVe weights

# GRU layer with return_sequences=True for compatibility with pooling
model.add(Bidirectional(GRU(64, return_sequences=True)))

# Dropout after GRU layer
model.add(Dropout(0.5))  # after GRU

# Global Max Pooling layer for dimensionality reduction
model.add(GlobalMaxPool1D())

# Dropout to prevent overfitting
model.add(Dropout(0.5))

# Output layer for classification into 5 classes (rating from 1 to 5)
model.add(Dense(5, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

# Model summary
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │      71,846,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_1               │ ?                           │     0 (unbuilt) │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 71,846,000 (274.07 MB)

 Trainable params: 71,846,000 (274.07 MB)

 Non-trainable params: 0 (0.00 B)

In [74]:

# Train the model
history = model.fit( X_train_first, y_train_first, validation_data=(X_val,y_val), epochs=3, batch_size=256, verbose=1)



Epoch 1/3
469/469 ━━━━━━━━━━━━━━━━━━━━ 45s 86ms/step - accuracy: 0.3737 - loss: 1.3943 - val_accuracy: 0.4988 - val_loss: 1.2100
Epoch 2/3
469/469 ━━━━━━━━━━━━━━━━━━━━ 41s 85ms/step - accuracy: 0.5549 - loss: 1.0418 - val_accuracy: 0.5036 - val_loss: 1.1586
Epoch 3/3
469/469 ━━━━━━━━━━━━━━━━━━━━ 41s 85ms/step - accuracy: 0.6395 - loss: 0.8709 - val_accuracy: 0.4870 - val_loss: 1.1753


In [75]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}") #0.001 , bidirectional GRU and 3 epochs

3000/3000 ━━━━━━━━━━━━━━━━━━━━ 28s 9ms/step - accuracy: 0.4914 - loss: 1.1742
Test Accuracy: 0.48734375834465027


In [77]:
#stpe severn
# Train the model
history = model.fit( X_train_second, y_train_second, validation_data=(X_val,y_val), epochs=5, batch_size=256, verbose=1,initial_epoch = 3)



Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 43s 91ms/step - accuracy: 0.5008 - loss: 1.1640 - val_accuracy: 0.5243 - val_loss: 1.1353
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 40s 86ms/step - accuracy: 0.6025 - loss: 0.9428 - val_accuracy: 0.5159 - val_loss: 1.1093


In [78]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}") #0.001 , bidirectional GRU and 3 epochs

3000/3000 ━━━━━━━━━━━━━━━━━━━━ 31s 10ms/step - accuracy: 0.5254 - loss: 1.1067
Test Accuracy: 0.5224791765213013


In [79]:
#stpe severn
# Train the model
history = model.fit( X_train_third, y_train_third, validation_data=(X_val,y_val), epochs=7, batch_size=256, verbose=1,initial_epoch = 5)



Epoch 6/7
469/469 ━━━━━━━━━━━━━━━━━━━━ 40s 86ms/step - accuracy: 0.5230 - loss: 1.1093 - val_accuracy: 0.5437 - val_loss: 1.1015
Epoch 7/7
469/469 ━━━━━━━━━━━━━━━━━━━━ 41s 86ms/step - accuracy: 0.6102 - loss: 0.9209 - val_accuracy: 0.5325 - val_loss: 1.0890


In [80]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}") #0.001 , bidirectional GRU and 3 epochs

3000/3000 ━━━━━━━━━━━━━━━━━━━━ 27s 9ms/step - accuracy: 0.5331 - loss: 1.0853
Test Accuracy: 0.5323958396911621


In [81]:
#stpe severn
# Train the model
history = model.fit( X_train_fourth, y_train_fourth, validation_data=(X_val,y_val), epochs=10, batch_size=256, verbose=1,initial_epoch = 5)



Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 40s 86ms/step - accuracy: 0.5292 - loss: 1.0925 - val_accuracy: 0.5413 - val_loss: 1.0919
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 40s 84ms/step - accuracy: 0.6189 - loss: 0.9063 - val_accuracy: 0.5318 - val_loss: 1.0758
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 41s 85ms/step - accuracy: 0.7080 - loss: 0.7182 - val_accuracy: 0.5235 - val_loss: 1.1106
Epoch 9/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 41s 84ms/step - accuracy: 0.7901 - loss: 0.5345 - val_accuracy: 0.5055 - val_loss: 1.2118
Epoch 10/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 42s 86ms/step - accuracy: 0.8491 - loss: 0.3926 - val_accuracy: 0.4912 - val_loss: 1.3335


In [82]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}") #0.001 , bidirectional GRU and 3 epochs

3000/3000 ━━━━━━━━━━━━━━━━━━━━ 31s 10ms/step - accuracy: 0.4927 - loss: 1.3232
Test Accuracy: 0.49159374833106995


In [84]:
#stpe severn
# Train the model
history = model.fit( X_train_third, y_train_third, validation_data=(X_val,y_val), epochs=11, batch_size=256, verbose=1,initial_epoch = 10)





Epoch 11/11
469/469 ━━━━━━━━━━━━━━━━━━━━ 41s 88ms/step - accuracy: 0.6947 - loss: 0.7488 - val_accuracy: 0.5171 - val_loss: 1.1265
Epoch 1/12


KeyboardInterrupt: 

In [85]:
history = model.fit( X_train_first, y_train_first, validation_data=(X_val,y_val), epochs=12, batch_size=256, verbose=1,initial_epoch = 12)


Epoch 1/12
Epoch 2/12


KeyboardInterrupt: 

In [86]:

history = model.fit( X_train_second, y_train_second, validation_data=(X_val,y_val), epochs=13, batch_size=256, verbose=1,initial_epoch = 12)

Epoch 13/13
469/469 ━━━━━━━━━━━━━━━━━━━━ 41s 87ms/step - accuracy: 0.6112 - loss: 0.9250 - val_accuracy: 0.5277 - val_loss: 1.0889


In [87]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}") #0.001 , bidirectional GRU and 3 epochs

3000/3000 ━━━━━━━━━━━━━━━━━━━━ 29s 10ms/step - accuracy: 0.5321 - loss: 1.0809
Test Accuracy: 0.5304062366485596
